In [1]:
# 라이브러리 설치
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings; warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error


In [115]:
# contents= ['개발 · 프로그래밍', '프로그래밍 언어']
contents= ['데이터사이언스', '데이터']
data = pd.read_csv('preprocess_inflearn_data.csv', delimiter=',')
hwangjae_data = pd.read_csv('hwangjae_inflearn_data.csv', delimiter=',')
data = data.drop('Unnamed: 0', axis=1)

# 컬럼명 설정
data.columns = ['Index', 'Title', 'Content', 'Level', 'Tag', 'Category', 'Author']
hwangjae_data.columns = ['Index', 'Title', 'Content', 'Level', 'Tag', 'Category', 'Author']

course = pd.read_csv('preprocess_inflearn_data.csv', delimiter=',')
course

,Unnamed: 0,Index,Title,Content,Level,Tag,Category,Author
0,0,158,['프로그래밍 시작하기 : 파이썬 입문 (Inflearn Original)'],"['입문', '문법', '프로그래밍', '업무', '자동화', '파이썬', '분석'...",입문자,['Python'],"['BEST', '개발 · 프로그래밍', '프로그래밍 언어']",['인프런']
1,1,159,['우아한 고성능 프로그래밍 언어 Rust 입문 및 활용'],['웹어셈블리'],초급자,"['rust', 'webassembly']","['NEW', '개발 · 프로그래밍', '프로그래밍 언어']",['김대현']
2,2,160,['[인프런x코드캠프] 부트캠프에서 만든 고농축 백엔드 코스'],"['커뮤니케이션', '프로그래밍', '포트폴리오', '고난', '통신', '언어',...",입문자,"['NestJS', 'Node.js', 'JavaScript', 'Docker', ...","['BEST', '개발 · 프로그래밍', '백엔드']",['코드캠프']
3,3,161,['[게임 프로그래머 입문 올인원] C++ & 자료구조/알고리즘 & STL & 게임...,"['알고리즘', '프로그래밍', '게임', '수학', '서버', '자료구조']",입문자,"['C++', 'UE Blueprint', '게임수학', 'windows-api',...","['게임 개발', '게임 프로그래밍']",['Rookiss']
4,4,162,['실전! 스프링 부트와 JPA 활용1 - 웹 애플리케이션 개발'],"['설계', '자바', '스프링부트', '웹', '실무', '개발', '최적', '...",초급자,"['Java', 'Spring', '웹앱', 'Spring Boot', 'JPA']","['BEST', '개발 · 프로그래밍', '백엔드']",['김영한']
...,...,...,...,...,...,...,...,...
2504,2566,2662,['[원포인트 MBA] 연세대 권구혁 교수의 경영전략'],"['경영', '핵심', '비즈니스', '트렌드', '이론']",초급자,['경영노하우'],"['비즈니스 · 마케팅', '경영']",['멀티캠퍼스']
2505,2567,2663,['워드프레스 속성 마스터 과정'],"['효과', '포인트', '전략', '마스터', '관점', '핵심', '기반', '...",초급자,['WordPress'],"['개발 · 프로그래밍', '웹 개발']",['김명준']
2506,2568,2664,['제발 도커 씁시다!'],['이유'],초급자,['Docker'],"['개발 · 프로그래밍', '데브옵스 · 인프라']",['에피']
2507,2569,2665,['엑셀 기초부터 고급까지'],"['엑셀', '자격증']",입문자,"['Excel', 'MS-Office']","['비즈니스 · 마케팅', '오피스']",['루미의 컴퓨터 강좌']


In [116]:

# content 기준
count_vect = CountVectorizer(min_df = 1, ngram_range=(1, 2))
content_mat = count_vect.fit_transform(course['Content'])
content_sim = cosine_similarity(content_mat, content_mat)
content_sim_sorted_ind = content_sim.argsort()[:, ::-1]


In [117]:
content_sim_sorted_ind[0]

array([   0, 1037, 1512, ..., 1525, 1529, 1254], dtype=int64)

In [118]:
def find_sim_course(df, sorted_ind, content_names, top_n = 10):
# title 칼럼이 입력된 title_name 값인 DataFrame추출
    content_course = df[df['Content'].apply(lambda x: any(content in x for content in content_names))]
    print(content_names)
    print(content_course)
    content_index = content_course.index.values
    similar_indexes = sorted_ind[content_index, :(top_n)]
    similar_indexes = similar_indexes.reshape(-1)
    return df.iloc[similar_indexes]

similar_course = find_sim_course(course, content_sim_sorted_ind, contents, 5)
similar_course_list = []
course["Content"] = hwangjae_data["Content"]

# similar_course_list = {"Title" : similar_course['Title'][:10].values.tolist(), "Content" : similar_course[["Content"]][:10].values.tolist(), "Tag" : similar_course[['Tag']][:10].values.tolist(), "Author" : similar_course[['Author']][:10].values.tolist()}
similar_course_list = {"MISSION_NO" : similar_course['Index'][:10].values.tolist()}
print(similar_course_list)


['데이터사이언스', '데이터']
      Unnamed: 0  Index                                              Title  \
0              0    158        ['프로그래밍 시작하기 : 파이썬 입문 (Inflearn Original)']   
12            12    170  ['처음하는 SQL과 데이터베이스(MySQL) 부트캠프 [입문부터 활용까지] (20...   
13            13    171                        ['데이터 분석 SQL Fundamentals']   
33            35    191  ['파이썬입문과 크롤링기초 부트캠프 [파이썬, 웹, 데이터 이해 기본까지] (202...   
43            45    201                    ['카프카 완벽 가이드 - 커넥트(Connect) 편']   
...          ...    ...                                                ...   
2461        2522   2619                            ['로그프레소 엔터프라이즈 - 대시보드']   
2463        2524   2621                  ['로그프레소 엔터프라이즈 - 실시간 처리 및 외부 연동']   
2464        2525   2622                           ['로그프레소 엔터프라이즈 - 배치 처리']   
2465        2526   2623                                     ['로그프레소 시작하기']   
2475        2536   2633                ['IBM Cloud를 이용한 무료 마인크래프트 서버 만들기']   

                                            

In [119]:
course

,Unnamed: 0,Index,Title,Content,Level,Tag,Category,Author
0,0,158,['프로그래밍 시작하기 : 파이썬 입문 (Inflearn Original)'],"['입문자도 쉽게 할 수 있는 프로그래밍 입문', '파이썬 기초 문법과 활용법을 배...",입문자,['Python'],"['BEST', '개발 · 프로그래밍', '프로그래밍 언어']",['인프런']
1,1,159,['우아한 고성능 프로그래밍 언어 Rust 입문 및 활용'],"['Rust', '웹어셈블리']",초급자,"['rust', 'webassembly']","['NEW', '개발 · 프로그래밍', '프로그래밍 언어']",['김대현']
2,2,160,['[인프런x코드캠프] 부트캠프에서 만든 고농축 백엔드 코스'],"['실무 포트폴리오 2개', '최신 기술 스택을 활용한 6개의 프로젝트 구현', '...",입문자,"['NestJS', 'Node.js', 'JavaScript', 'Docker', ...","['BEST', '개발 · 프로그래밍', '백엔드']",['코드캠프']
3,3,161,['[게임 프로그래머 입문 올인원] C++ & 자료구조/알고리즘 & STL & 게임...,"['C++', '게임 프로그래밍', '자료구조 & 알고리즘', 'STL', '게임 ...",입문자,"['C++', 'UE Blueprint', '게임수학', 'windows-api',...","['게임 개발', '게임 프로그래밍']",['Rookiss']
4,4,162,['실전! 스프링 부트와 JPA 활용1 - 웹 애플리케이션 개발'],['스프링 부트와 JPA를 활용해서 실무에서 자바 웹 애플리케이션을 개발할 수 있습...,초급자,"['Java', 'Spring', '웹앱', 'Spring Boot', 'JPA']","['BEST', '개발 · 프로그래밍', '백엔드']",['김영한']
...,...,...,...,...,...,...,...,...
2504,2566,2662,['[원포인트 MBA] 연세대 권구혁 교수의 경영전략'],"['Microsoft 협업 서비스의 이해', 'Teams 새로운 기능', 'Team...",초급자,['경영노하우'],"['비즈니스 · 마케팅', '경영']",['멀티캠퍼스']
2505,2567,2663,['워드프레스 속성 마스터 과정'],"['쿠팡, 구글 무비, 네이버 등 각종 사이트 스크래핑 전략', '동적으로 로딩되는...",초급자,['WordPress'],"['개발 · 프로그래밍', '웹 개발']",['김명준']
2506,2568,2664,['제발 도커 씁시다!'],"['OneNote 2013의 주요 기능', '클라우드 환경에 최적화된 OneNote...",초급자,['Docker'],"['개발 · 프로그래밍', '데브옵스 · 인프라']",['에피']
2507,2569,2665,['엑셀 기초부터 고급까지'],"['Access 2013을 이용하여 데이터베이스를 구축하는 방법', '단순 혹은 반...",입문자,"['Excel', 'MS-Office']","['비즈니스 · 마케팅', '오피스']",['루미의 컴퓨터 강좌']
